In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import gc


from sklearn.svm import SVC, NuSVC
from sklearn import preprocessing
from sklearn import impute
from sklearn import pipeline

import optuna
from optuna.visualization import (
    plot_contour
    , plot_edf
    , plot_intermediate_values
    , plot_optimization_history
    , plot_parallel_coordinate
    , plot_param_importances
    , plot_slice
)

pd.set_option('display.max_columns', None)


import plotly.express as px
import plotly.io as pio
pio.renderers.default = "png"

/home/mavillan/mambaforge/envs/kg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***
## loading data

In [2]:
input_path = "../data/raw"

train = pd.read_csv(f"{input_path}/train.csv")
test  = pd.read_csv(f"{input_path}/test.csv")
greeks = pd.read_csv(f"{input_path}/greeks.csv")

train.columns = [col.strip() for col in train.columns]
test.columns = [col.strip() for col in test.columns]

# available features
input_cols = train.columns[1:-1]
categ_cols = ["EJ"]

# we extend train with dummies from greeks
dummies = pd.get_dummies(greeks[["Alpha","Beta","Gamma","Delta"]])
train[dummies.columns] = dummies

# encode of categorical features
encoder = preprocessing.LabelEncoder().fit(train["EJ"])
train["EJ"] = encoder.transform(train["EJ"]).astype(int)
test["EJ"] = encoder.transform(test["EJ"]).astype(int)

display(train)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class,Alpha_A,Alpha_B,Alpha_D,Alpha_G,Beta_A,Beta_B,Beta_C,Gamma_A,Gamma_B,Gamma_E,Gamma_F,Gamma_G,Gamma_H,Gamma_M,Gamma_N,Delta_A,Delta_B,Delta_C,Delta_D
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.238680,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,1,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.238680,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,0,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.238680,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,1,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.238680,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,1,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.238680,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,1,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.68439,21.1860,167.877117,27.287375,365.516874,257.432377,41.368691,0.691257,55.163024,4.780452,0.013930,1.177525,0.698250,40.159779,1.070298,7.030640,21.75904,355.930925,0.238680,0.445479,176.977590,90.91832,27.957928,0.005518,2.41906,32.508604,8.015112,1.354416,495.086300,0.003042,0,51.

In [3]:
preproc_pipe = pipeline.Pipeline([
    ("imputer", impute.SimpleImputer(strategy="median")), 
    ("scaler", preprocessing.MaxAbsScaler()),
])

preproc_pipe.fit(train[input_cols])
display(preproc_pipe)

train[input_cols] = preproc_pipe.transform(train[input_cols])
test[input_cols] = preproc_pipe.transform(test[input_cols])

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MaxAbsScaler())])

In [4]:
repeated_cv_split = joblib.load("../data/iarc-data-split/repeated_5fold_cv_split_4tuning.pkl")
print(len(repeated_cv_split))

# number of repetitions to use
REPETITIONS = 10

10


***
## training

In [5]:
def balanced_logloss_(y_pred, y_true):
    n0 = np.sum(1-y_true)
    n1 = np.sum(y_true)
    p1 = np.clip(y_pred, 1e-15, 1-1e-15)
    p0 = 1-p1
    log_loss0 = - np.sum((1-y_true) * np.log(p0)) / n0
    log_loss1 = - np.sum(y_true * np.log(p1)) / n1
    return (log_loss0 + log_loss1)/2

#def balanced_logloss(y_pred: np.ndarray, data: lgb.Dataset):
#    y_true = data.get_label()
#    return 'balanced_logloss', balanced_logloss_(y_pred, y_true), False 

In [6]:
pct = train.Class.value_counts(normalize=True)
scale_pos_weight = pct[0]/pct[1]
print("scale_pos_weight:", scale_pos_weight)

cnt = train.Class.value_counts(normalize=False)
neg_bagging_fraction = cnt[1]/cnt[0]
print("neg_bagging_fraction:", neg_bagging_fraction)

scale_pos_weight: 4.712962962962963
neg_bagging_fraction: 0.21218074656188604


In [7]:
DEFAULT_PARAMS = dict(
    probability=True,
    cache_size=500,
    verbose=False,
    max_iter=-1,
    random_state=2112,
    nu=0.3
)

def train_validate(
        dataframe,
        input_cols, 
        model_params,
        repeated_cv_split,
        n_repetitions=REPETITIONS,
        verbose=False,
    ):

    metrics = list()
    model_params = dict(model_params)

    for repeat in range(n_repetitions):
        if verbose:
            print(f"REPEAT NUMBER: {repeat+1}/{n_repetitions}")
        cv_split = repeated_cv_split[f"repeat_{repeat}"]
        n_folds = len(cv_split)
        
        for split in cv_split:
            fold = split["fold"]
            train_idx = split["train_idx"]
            valid_idx = split["valid_idx"]
            if verbose:
                print(f"training model for fold: {fold+1}/{n_folds}")
        
            train_df = dataframe.loc[train_idx,:].reset_index(drop=True)
            valid_df = dataframe.loc[valid_idx,:].reset_index(drop=True)

            try:
                clf = NuSVC(**model_params)
                clf.fit(
                    train_df[input_cols].values, 
                    train_df["Class"].values, 
                )
                y_pred = clf.predict_proba(valid_df[input_cols].values)
                metrics.append( balanced_logloss_(y_pred[:,1], valid_df["Class"].values) )
            except:
                metrics.append(10.)
    
    return np.mean(metrics), np.std(metrics)


def objective(trial):
    _model_params = dict(
        nu = trial.suggest_float("nu", 1e-15, 0.3),
        kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        degree = trial.suggest_int("degree", 2, 5),
        gamma = trial.suggest_float("gamma", 1e-5, 1e2, log=True),
        coef0 = trial.suggest_float("coef0", -1, 1),
        shrinking = trial.suggest_categorical("shrinking", [True, False]),
        tol = 10**(trial.suggest_int("tol_exp", -8, -2)),
        class_weight = trial.suggest_categorical("class_weight", ["balanced", None]),        
    )
    model_params = {**DEFAULT_PARAMS, **_model_params}
    
    metric_mean, metric_std = train_validate(
        dataframe = train,
        input_cols = input_cols,
        model_params = model_params,
        repeated_cv_split = repeated_cv_split,
        n_repetitions = REPETITIONS,
        verbose = False,
    )
    
    return metric_mean

In [8]:
%%time
train_validate(
    dataframe = train,
    input_cols = input_cols,
    model_params = DEFAULT_PARAMS,
    repeated_cv_split = repeated_cv_split,
    n_repetitions = REPETITIONS,
    verbose = False,
)

del DEFAULT_PARAMS["nu"]

CPU times: user 3.72 s, sys: 0 ns, total: 3.72 s
Wall time: 3.73 s


In [9]:
do_optimize = True

study = optuna.create_study(
    study_name="iarc-nu-svc",
    direction='minimize',
    storage='sqlite:///iarc-nu-svc.db',
    load_if_exists=True,
)

if do_optimize:
    study.optimize(
        objective, 
        n_trials=5000, 
        timeout=43200, # 12 hours
        n_jobs=1, 
        gc_after_trial=True,
    ) 

[I 2023-07-12 05:28:35,454] Using an existing study with name 'iarc-nu-svc' instead of creating a new one.


[I 2023-07-12 05:28:42,521] Trial 5 finished with value: 0.646337872942147 and parameters: {'class_weight': 'balanced', 'coef0': 0.9100187449752752, 'degree': 4, 'gamma': 7.206333334273951, 'kernel': 'rbf', 'nu': 0.2375154152133495, 'shrinking': False, 'tol_exp': -3}. Best is trial 0 with value: 0.5453918839749361.


[I 2023-07-12 05:28:51,030] Trial 6 finished with value: 0.7899176285508966 and parameters: {'class_weight': None, 'coef0': 0.09226253044085464, 'degree': 4, 'gamma': 14.88387335792666, 'kernel': 'rbf', 'nu': 0.13698054914385313, 'shrinking': False, 'tol_exp': -8}. Best is trial 0 with value: 0.5453918839749361.


[I 2023-07-12 05:28:57,872] Trial 7 finished with value: 0.579925618013994 and parameters: {'class_weight': None, 'coef0': 0.49836577164714835, 'degree': 3, 'gamma': 4.347502654986356, 'kernel': 'rbf', 'nu': 0.23916605549760672, 'shrinking': True, 'tol_exp': -3}. Best is trial 0 with value: 0.5453918839749361.


[I 2023-07-12 05:29:01,586] Trial 8 finished with value: 0.5937216165164305 and parameters: {'class_weight': 'balanced', 'coef0': -0.8554705734585533, 'degree': 3, 'gamma': 1.2517537645546045e-05, 'kernel': 'sigmoid', 'nu': 0.2960279602812321, 'shrinking': False, 'tol_exp': -6}. Best is trial 0 with value: 0.5453918839749361.


...

[I 2023-07-12 14:51:52,021] Trial 762 finished with value: 0.6309260743046455 and parameters: {'class_weight': None, 'coef0': -0.17859733701636143, 'degree': 3, 'gamma': 6.4628616380307715, 'kernel': 'rbf', 'nu': 0.17294164492460415, 'shrinking': False, 'tol_exp': -7}. Best is trial 253 with value: 0.442635606472385.


[I 2023-07-12 14:52:00,939] Trial 763 finished with value: 0.5641521550908346 and parameters: {'class_weight': None, 'coef0': 0.3074408443368303, 'degree': 3, 'gamma': 0.00022332450226934382, 'kernel': 'rbf', 'nu': 0.16433992217340201, 'shrinking': False, 'tol_exp': -7}. Best is trial 253 with value: 0.442635606472385.


[I 2023-07-12 14:52:08,631] Trial 764 finished with value: 0.4967203785276703 and parameters: {'class_weight': None, 'coef0': -0.01586648010447933, 'degree': 3, 'gamma': 0.1821530312301654, 'kernel': 'rbf', 'nu': 0.08835762771745911, 'shrinking': False, 'tol_exp': -7}. Best is trial 253 with value: 0.442635606472385.


[I 2023-07-12 14:52:21,651] Trial 765 finished with value: 0.5326202089350618 and parameters: {'class_weight': None, 'coef0': -0.8987240945783866, 'degree': 3, 'gamma': 0.002539386006914562, 'kernel': 'rbf', 'nu': 0.18355656039717905, 'shrinking': False, 'tol_exp': -8}. Best is trial 253 with value: 0.442635606472385.


[I 2023-07-12 14:52:27,418] Trial 766 finished with value: 0.4654211564045675 and parameters: {'class_weight': None, 'coef0': -0.8415606000582551, 'degree': 3, 'gamma': 0.5498936926973026, 'kernel': 'rbf', 'nu': 0.12175416848180827, 'shrinking': False, 'tol_exp': -7}. Best is trial 253 with value: 0.442635606472385.


In [ ]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_edf(study)

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
best_params = dict(study.best_params)
best_params["tol"] = 10**best_params.pop("tol_exp")
best_params

***